# Week 3: Filtering & Frequency Domain — Lab Practice

**Topics:** Image Resizing, Convolution, Smoothing, Frequency Domain & Sharpening

This notebook accompanies the Week 3 lecture slides. We will resize images with different interpolation methods, implement 2D convolution from scratch, apply smoothing and sharpening filters, and visualize the frequency domain — all hands-on.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from scipy.ndimage import correlate
from scipy.signal import convolve2d

%matplotlib inline

### Environment setup

This notebook works both **locally** and on **Google Colab**.
- **Local**: images are loaded from the repository’s `images/` folder.
- **Colab**: images are automatically downloaded from GitHub on first run.

In [ ]:
import os, urllib.request

# Detect Google Colab
IN_COLAB = "google.colab" in str(get_ipython()) if hasattr(__builtins__, "__IPYTHON__") else False

# Image paths
REPO_URL = "https://raw.githubusercontent.com/HyeongminLEE/image-processing-tutorial/main"
IMAGE_DIR = "images"
IMAGE_NAME = "parrots_square.jpg"

if IN_COLAB:
    os.makedirs(IMAGE_DIR, exist_ok=True)
    url = f"{REPO_URL}/{IMAGE_DIR}/{IMAGE_NAME}"
    dest = os.path.join(IMAGE_DIR, IMAGE_NAME)
    if not os.path.exists(dest):
        print(f"Downloading {IMAGE_NAME} ...")
        urllib.request.urlretrieve(url, dest)
        print("Done.")
    IMAGE_PATH = dest
else:
    IMAGE_PATH = os.path.join("..", IMAGE_DIR, IMAGE_NAME)

# Grayscale image (saved in Week 2; regenerate if missing)
GRAY_PATH = os.path.join("..", "week2", "parrots_gray.jpg")
if not os.path.exists(GRAY_PATH):
    GRAY_PATH = "parrots_gray.jpg"
    _gray = np.array(Image.open(IMAGE_PATH).convert("L"))
    Image.fromarray(_gray).save(GRAY_PATH)
    print(f"Generated grayscale image at {GRAY_PATH}")

print(f"Image path: {IMAGE_PATH}")
print(f"Running on: {'Google Colab' if IN_COLAB else 'Local'}")

### Display helpers

We define utility functions used throughout this notebook. This week adds `show_kernel` for visualizing filter weights and `show_spectrum` for frequency domain analysis, alongside the familiar `show_images`.

In [ ]:
def show_images(*imgs, titles=None, scale=4):
    """Display images in a single row.

    Grayscale (2-D) arrays automatically use a gray colormap.
    *titles* is an optional list of strings, one per image.
    """
    n = len(imgs)
    fig, axes = plt.subplots(1, n, figsize=(scale * n, scale))
    if n == 1:
        axes = [axes]
    for i, (ax, img) in enumerate(zip(axes, imgs)):
        if img.ndim == 2:
            ax.imshow(img, cmap="gray", vmin=0, vmax=255)
        else:
            ax.imshow(img)
        if titles:
            ax.set_title(titles[i])
        ax.axis("off")
    plt.tight_layout()
    plt.show()


def show_proportional(*imgs, titles=None, scale=6):
    """Display images vertically, each at its proportional size.

    *scale* sets the width (in inches) of the widest image.
    """
    n = len(imgs)
    max_w = max(img.shape[1] for img in imgs)
    px_per_inch = max_w / scale
    heights = [img.shape[0] / px_per_inch for img in imgs]
    widths = [img.shape[1] / px_per_inch for img in imgs]
    total_h = sum(heights) + 0.5 * (n - 1)

    fig, axes = plt.subplots(n, 1, figsize=(scale, total_h),
                             gridspec_kw={"height_ratios": [img.shape[0] for img in imgs]})
    if n == 1:
        axes = [axes]
    for i, (ax, img) in enumerate(zip(axes, imgs)):
        ax.set_anchor("W")          # left-align each subplot
        if img.ndim == 2:
            ax.imshow(img, cmap="gray", vmin=0, vmax=255)
        else:
            ax.imshow(img)
        if titles:
            ax.set_title(titles[i], loc="left")
        ax.axis("off")
    fig.subplots_adjust(hspace=0.05)
    plt.show()


def show_kernel(kernel, title="Kernel", ax=None):
    """Display a small kernel as an annotated heatmap."""
    standalone = ax is None
    if standalone:
        fig, ax = plt.subplots(figsize=(3, 3))
    vmax = np.max(np.abs(kernel))
    ax.imshow(kernel, cmap="coolwarm", vmin=-vmax, vmax=vmax)
    for i in range(kernel.shape[0]):
        for j in range(kernel.shape[1]):
            val = kernel[i, j]
            text = f"{val:.2f}" if val != int(val) else str(int(val))
            ax.text(j, i, text, ha="center", va="center", fontsize=10)
    ax.set_title(title)
    ax.set_xticks([])
    ax.set_yticks([])
    if standalone:
        plt.tight_layout()
        plt.show()


def show_spectrum(img, title=""):
    """Display a grayscale image and its centered log-magnitude spectrum side by side."""
    F = np.fft.fft2(img.astype(np.float64))
    F_shifted = np.fft.fftshift(F)
    magnitude = np.log1p(np.abs(F_shifted))

    fig, axes = plt.subplots(1, 2, figsize=(10, 4))
    axes[0].imshow(img, cmap="gray", vmin=0, vmax=255)
    axes[0].set_title(f"{title} (Spatial)" if title else "Spatial domain")
    axes[0].axis("off")
    axes[1].imshow(magnitude, cmap="gray")
    axes[1].set_title(f"{title} (Spectrum)" if title else "Magnitude spectrum")
    axes[1].axis("off")
    plt.tight_layout()
    plt.show()

---
## 1. Image Resizing & Interpolation

When we change an image’s dimensions, pixels must be **created** (upsampling) or **discarded** (downsampling). The interpolation method determines how new pixel values are estimated from existing ones.

In [ ]:
img_color = np.array(Image.open(IMAGE_PATH))
img_gray = np.array(Image.open(GRAY_PATH))

print(f"Color \u2014 shape: {img_color.shape}, dtype: {img_color.dtype}")
print(f"Gray  \u2014 shape: {img_gray.shape}, dtype: {img_gray.dtype}")

### Downsampling by pixel skipping

The simplest downsampling: take every $N$th pixel and discard the rest. This is just NumPy slicing: `img[::N, ::N]`.

Each image below is displayed at its **actual relative size** so you can see how much information is lost.

In [ ]:
factors = [1, 2, 4, 8]
downsampled = [img_gray[::f, ::f] for f in factors]
titles = [f"{d.shape[0]}×{d.shape[1]}" if f == 1
          else f"{d.shape[0]}×{d.shape[1]} (1/{f})"
          for f, d in zip(factors, downsampled)]

show_proportional(*downsampled, titles=titles)

### Upsampling with different interpolation methods

To upsample, we must **estimate** new pixel values. Three common methods:
- **Nearest neighbor**: copy the closest pixel — fast but blocky
- **Bilinear**: weighted average of 4 neighbors — smooth but slightly blurry
- **Bicubic**: cubic polynomial through 16 neighbors — sharpest, most common default

Let’s downsample to a small size, then upsample back to see the differences.

In [ ]:
pil_gray = Image.open(GRAY_PATH)
H, W = img_gray.shape

# Downsample to 64\u00d764
small = pil_gray.resize((64, 64), Image.NEAREST)

# Upsample back with different methods
methods = {"Nearest": Image.NEAREST, "Bilinear": Image.BILINEAR, "Bicubic": Image.BICUBIC}
upsampled = {name: np.array(small.resize((W, H), method)) for name, method in methods.items()}

show_images(img_gray, upsampled["Nearest"], upsampled["Bilinear"], upsampled["Bicubic"],
            titles=["Original", "Nearest", "Bilinear", "Bicubic"])

Let’s zoom in on a small region to see the interpolation artifacts more clearly.

In [ ]:
r1, r2, c1, c2 = 150, 300, 150, 300
crops = [img_gray[r1:r2, c1:c2]] + [upsampled[m][r1:r2, c1:c2] for m in methods]

show_images(*crops, titles=["Original"] + list(methods.keys()), scale=3)

### Exercises

**Exercise 1.1:** Downsample the **color** image by a factor of 4 using pixel skipping (`img_color[::4, ::4]`), then upsample it back to the original size using PIL with `Image.BICUBIC`. Display the original and the result side by side. What details are lost?

*Steps:*
1. Downsample: `small = img_color[::4, ::4]`
2. Convert to PIL: `Image.fromarray(small)`
3. Resize back: `.resize((W, H), Image.BICUBIC)` where `H, W = img_color.shape[:2]`
4. Convert to array: `np.array(...)`

In [ ]:
# YOUR CODE HERE

**Exercise 1.2:** Compare the histograms of the original grayscale image and the nearest-neighbor upsampled version (from the demo above). Plot them overlaid on the same axes using `plt.fill_between`. What do you notice about the histogram shape?

*Hint:* This connects to **Week 2 Section 7 (Histograms)**. Compute histograms with `np.histogram(img.ravel(), 256, [0, 256])` and plot with `plt.fill_between`. The nearest-neighbor histogram should look like a “comb” pattern because only certain pixel values are present.

In [ ]:
# YOUR CODE HERE

---
## 2. Spatial Filtering & Convolution

In Week 2, we transformed each pixel **independently** (point processing: $g(x,y) = T[f(x,y)]$). Now we use **neighborhoods**: the output at each pixel depends on the pixel *and its surrounding region*.

The core operation is **convolution**: slide a small weight matrix (kernel) over the image, computing a weighted sum at each position.

$$g(x,y) = \sum_{s=-k}^{k}\sum_{t=-k}^{k} h(s,t) \cdot f(x-s,\, y-t)$$

### 1D convolution: warm-up

Before tackling 2D images, let’s see convolution on a 1D signal. The steps are:
1. **Flip** the kernel
2. **Slide** it across the signal
3. At each position, **multiply** element-wise and **sum**

In [ ]:
signal = np.array([0, 0, 1, 2, 3, 2, 1, 0, 0], dtype=np.float64)
kernel_1d = np.array([1, 0, -1], dtype=np.float64)

# Manual 1D convolution (valid mode)
out_len = len(signal) - len(kernel_1d) + 1
output = np.zeros(out_len)
k_flip = kernel_1d[::-1]
for i in range(out_len):
    output[i] = np.sum(signal[i:i + len(kernel_1d)] * k_flip)

fig, axes = plt.subplots(1, 3, figsize=(12, 3))
axes[0].stem(signal)
axes[0].set_title("Signal")
axes[1].stem(kernel_1d)
axes[1].set_title("Kernel [1, 0, -1]")
axes[2].stem(output)
axes[2].set_title("Convolution output (valid)")
plt.tight_layout()
plt.show()

### 2D convolution: step by step

Now let’s extend to 2D. We’ll work through a small example: a 5×5 “image” convolved with a 3×3 kernel.

In [ ]:
small_img = np.array([
    [1,  2,  3,  4,  5],
    [6,  7,  8,  9,  10],
    [11, 12, 13, 14, 15],
    [16, 17, 18, 19, 20],
    [21, 22, 23, 24, 25]
], dtype=np.float64)

kernel_2d = np.array([
    [0, -1,  0],
    [-1, 5, -1],
    [0, -1,  0]
], dtype=np.float64)

print(f"Image (5\u00d75):\n{small_img.astype(int)}")
print()
show_kernel(kernel_2d, title="Kernel (3\u00d73)")

Let’s compute one output pixel explicitly: extract the 3×3 neighborhood, flip the kernel 180°, multiply element-wise, and sum.

In [ ]:
# For convolution: flip the kernel 180 degrees
k_flip = kernel_2d[::-1, ::-1]

# Compute output at position (1, 1) \u2014 kernel centered at row=1, col=1
row, col = 1, 1
neighborhood = small_img[row:row + 3, col:col + 3]
products = neighborhood * k_flip

print(f"Neighborhood at ({row},{col}):\n{neighborhood.astype(int)}")
print(f"\nFlipped kernel:\n{k_flip.astype(int)}")
print(f"\nElement-wise products:\n{products.astype(int)}")
print(f"\nSum = {products.sum():.0f}")

### Implementing 2D convolution from scratch

Let’s write a full convolution function. This builds deep intuition for what every library function does under the hood.

In [ ]:
def conv2d(image, kernel, mode="same"):
    """2D convolution (not correlation) of a grayscale image with a kernel.

    Parameters
    ----------
    image  : 2D array (H, W)
    kernel : 2D array (kH, kW), must be odd-sized
    mode   : "valid" or "same"
    """
    img = image.astype(np.float64)
    k = kernel[::-1, ::-1].astype(np.float64)   # flip for true convolution
    kH, kW = k.shape

    if mode == "same":
        pH, pW = kH // 2, kW // 2
        img = np.pad(img, ((pH, pH), (pW, pW)), mode="constant", constant_values=0)
    elif mode != "valid":
        raise ValueError(f"Unknown mode: {mode}")

    H, W = img.shape
    outH, outW = H - kH + 1, W - kW + 1
    output = np.zeros((outH, outW), dtype=np.float64)

    for i in range(outH):
        for j in range(outW):
            output[i, j] = np.sum(img[i:i + kH, j:j + kW] * k)

    return output

In [ ]:
result_valid = conv2d(small_img, kernel_2d, mode="valid")
result_same = conv2d(small_img, kernel_2d, mode="same")

print(f"Input shape:        {small_img.shape}")
print(f"Kernel shape:       {kernel_2d.shape}")
print(f"Valid output shape: {result_valid.shape}  (5 - 3 + 1 = 3)")
print(f"Same output shape:  {result_same.shape}   (padded to match input)")
print(f"\nValid output:\n{result_valid.astype(int)}")
print(f"\nSame output:\n{result_same.astype(int)}")

### Convolution vs Correlation

**Convolution** flips the kernel 180° before sliding: $g(x,y) = \sum h(s,t) \cdot f(x\mathbf{-}s,\, y\mathbf{-}t)$

**Correlation** uses the kernel as-is: $g(x,y) = \sum h(s,t) \cdot f(x\mathbf{+}s,\, y\mathbf{+}t)$

For **symmetric** kernels (like box and Gaussian filters), they produce identical results. Most libraries (including `scipy.ndimage.correlate`) compute **correlation** but call it “convolution” by convention.

In [ ]:
# Our implementation (true convolution)
ours = conv2d(small_img, kernel_2d, mode="same")

# scipy.signal.convolve2d (true convolution)
scipy_conv = convolve2d(small_img, kernel_2d, mode="same", boundary="fill", fillvalue=0)

# scipy.ndimage.correlate (correlation)
scipy_corr = correlate(small_img, kernel_2d, mode="constant", cval=0)

print(f"Our conv2d:\n{ours.astype(int)}")
print(f"\nscipy convolve2d:\n{scipy_conv.astype(int)}")
print(f"\nscipy correlate:\n{scipy_corr.astype(int)}")
print(f"\nOurs matches convolve2d: {np.allclose(ours, scipy_conv)}")

### Applying convolution to a real image

Let’s apply the sharpening kernel to the grayscale parrot image.

> Since our from-scratch implementation uses Python loops, it is **slow** on full-size images. For the rest of this notebook we use `scipy.ndimage.correlate` for efficiency (for symmetric kernels, correlation = convolution).

In [ ]:
sharpened = correlate(img_gray.astype(np.float64), kernel_2d, mode="constant", cval=0)
sharpened = np.clip(sharpened, 0, 255).astype(np.uint8)

show_images(img_gray, sharpened, titles=["Original", "Sharpened (kernel from above)"])

### Output size

The output dimensions depend on the padding mode:

| Mode | Padding $p$ | Output size |
|:---:|:---:|:---:|
| **Valid** | $0$ | $H - F + 1$ |
| **Same** | $\lfloor F/2 \rfloor$ | $H$ |
| **Full** | $F - 1$ | $H + F - 1$ |

General formula: $O = H + 2p - F + 1$, where $F$ is the kernel size.

In [ ]:
H, W = img_gray.shape
F = 5
test_kernel = np.ones((F, F)) / (F * F)

valid_out = convolve2d(img_gray, test_kernel, mode="valid")
same_out  = convolve2d(img_gray, test_kernel, mode="same")
full_out  = convolve2d(img_gray, test_kernel, mode="full")

print(f"Input:  {img_gray.shape}")
print(f"Kernel: {test_kernel.shape}")
print(f"Valid:  {valid_out.shape}  (expected {H - F + 1}\u00d7{W - F + 1})")
print(f"Same:   {same_out.shape}  (expected {H}\u00d7{W})")
print(f"Full:   {full_out.shape}  (expected {H + F - 1}\u00d7{W + F - 1})")

### Exercises

**Exercise 2.1:** Implement a **correlation** function `corr2d(image, kernel, mode="same")` by modifying the `conv2d` function above. The only difference: do **not** flip the kernel.

Test it on `small_img` with `kernel_2d` and verify that:
- Your `corr2d` matches `scipy.ndimage.correlate`
- For the **symmetric** identity kernel $\begin{bmatrix}0&0&0\\0&1&0\\0&0&0\end{bmatrix}$, both `conv2d` and `corr2d` produce the same result (the original image)

*Hint:* Copy `conv2d`, remove the line `k = kernel[::-1, ::-1]...`, and use the kernel directly.

In [ ]:
# YOUR CODE HERE

**Exercise 2.2:** Use `scipy.ndimage.correlate` to apply the **identity kernel** $\begin{bmatrix}0&0&0\\0&1&0\\0&0&0\end{bmatrix}$ to the grayscale image. Display the original and the result side by side — they should look identical.

Then apply a **shift kernel** $\begin{bmatrix}0&0&0\\0&0&0\\0&0&1\end{bmatrix}$ and observe the effect. What happens to the image?

*Hint:* Create the identity kernel with `K = np.array([[0,0,0],[0,1,0],[0,0,0]])`. The shift kernel places the `1` at a different position, which shifts the image.

In [ ]:
# YOUR CODE HERE

---
## 3. Smoothing Filters

Smoothing filters blur the image by averaging each pixel with its neighbors. They are **low-pass** filters: they preserve gradual intensity changes (low frequencies) and suppress rapid changes like edges and noise (high frequencies).

Two main types:
- **Box filter**: all weights equal (simple average)
- **Gaussian filter**: closer neighbors get higher weight (bell curve)

### Box filter

The simplest smoothing kernel: every weight is $\frac{1}{N^2}$. A 3×3 box filter:

$$h = \frac{1}{9}\begin{bmatrix}1 & 1 & 1\\1 & 1 & 1\\1 & 1 & 1\end{bmatrix}$$

Each output pixel becomes the **average** of its $3 \times 3$ neighborhood.

In [ ]:
def make_box_kernel(size):
    """Create a box (mean) kernel of the given size."""
    return np.ones((size, size), dtype=np.float64) / (size * size)

box3 = make_box_kernel(3)
show_kernel(box3, title="3\u00d73 Box kernel")

In [ ]:
sizes = [3, 7, 15, 31]
blurred = []
for s in sizes:
    k = make_box_kernel(s)
    b = correlate(img_gray.astype(np.float64), k, mode="constant", cval=0)
    blurred.append(np.clip(b, 0, 255).astype(np.uint8))

show_images(img_gray, *blurred,
            titles=["Original"] + [f"Box {s}\u00d7{s}" for s in sizes])

### Gaussian filter

Instead of equal weights, a Gaussian kernel gives **higher weight to closer neighbors**:

$$h(s,t) = \frac{1}{2\pi\sigma^2}\exp\!\left(-\frac{s^2+t^2}{2\sigma^2}\right)$$

The parameter $\sigma$ controls the spread: larger $\sigma$ = wider kernel = more smoothing.

In [ ]:
def make_gaussian_kernel(size, sigma):
    """Create a 2D Gaussian kernel of the given size and sigma."""
    k = size // 2
    ax = np.arange(-k, k + 1, dtype=np.float64)
    xx, yy = np.meshgrid(ax, ax)
    kernel = np.exp(-(xx**2 + yy**2) / (2 * sigma**2))
    kernel /= kernel.sum()
    return kernel

g5 = make_gaussian_kernel(5, sigma=1.0)
print(f"Kernel shape: {g5.shape}")
print(f"Sum of weights: {g5.sum():.6f}")
show_kernel(g5, title="5\u00d75 Gaussian (\u03c3=1.0)")

In [ ]:
g15 = make_gaussian_kernel(15, sigma=3.0)
ax_range = np.arange(-7, 8)
xx, yy = np.meshgrid(ax_range, ax_range)

fig = plt.figure(figsize=(6, 5))
ax = fig.add_subplot(111, projection="3d")
ax.plot_surface(xx, yy, g15, cmap="viridis")
ax.set_title("Gaussian kernel (\u03c3=3.0)")
ax.set_xlabel("x")
ax.set_ylabel("y")
plt.tight_layout()
plt.show()

In [ ]:
sigmas = [1, 3, 5, 10]
gaussian_blurred = []
for sigma in sigmas:
    size = int(6 * sigma + 1) | 1   # ensure odd size, ~6\u03c3 wide
    k = make_gaussian_kernel(size, sigma)
    b = correlate(img_gray.astype(np.float64), k, mode="constant", cval=0)
    gaussian_blurred.append(np.clip(b, 0, 255).astype(np.uint8))

show_images(img_gray, *gaussian_blurred,
            titles=["Original"] + [f"Gaussian \u03c3={s}" for s in sigmas])

### Box vs Gaussian comparison

At similar kernel sizes, Gaussian produces a **smoother, more natural** blur because nearby pixels contribute more than distant ones.

In [ ]:
box_result = correlate(img_gray.astype(np.float64), make_box_kernel(15), mode="constant", cval=0)
gauss_result = correlate(img_gray.astype(np.float64), make_gaussian_kernel(15, 3.0), mode="constant", cval=0)

box_result = np.clip(box_result, 0, 255).astype(np.uint8)
gauss_result = np.clip(gauss_result, 0, 255).astype(np.uint8)

show_images(img_gray, box_result, gauss_result,
            titles=["Original", "Box 15\u00d715", "Gaussian 15\u00d715 (\u03c3=3)"])

### Exercises

**Exercise 3.1:** Apply a 7×7 box filter to the **color** image. Process each channel (R, G, B) independently, then recombine into a color image.

*Steps:*
1. Split channels: `r, g, b = img_color[:,:,0], img_color[:,:,1], img_color[:,:,2]`
2. Apply `correlate` to each channel (cast to `float64`, use `make_box_kernel(7)`)
3. Clip each to [0, 255] and cast to `uint8`
4. Stack: `result = np.stack([r_blur, g_blur, b_blur], axis=2)`
5. Display original and blurred with `show_images`

In [ ]:
# YOUR CODE HERE

**Exercise 3.2:** Smooth the grayscale image with a Gaussian filter ($\sigma=3$), then compute the **difference** between the original and the smoothed image: `diff = original - blurred`. This difference is the **high-frequency detail** that smoothing removes.

Display three images: original, smoothed, and the difference (rescaled for visibility).

*Hint:* The difference image may contain negative values. To display it, rescale to [0, 255]:
```python
diff = img_gray.astype(np.float64) - blurred.astype(np.float64)
diff_display = ((diff - diff.min()) / (diff.max() - diff.min()) * 255).astype(np.uint8)
```

This is the same min-max rescaling from **Week 2 Section 8 (Histogram Stretching)** — we stretch the difference image to the full [0, 255] range for display. This difference image also foreshadows the sharpening concept in Section 5.

In [ ]:
# YOUR CODE HERE

---
## 4. Frequency Domain & 2D FFT

The **Convolution Theorem** says: convolution in the spatial domain equals **multiplication** in the frequency domain.

$$f * h \;\longleftrightarrow\; F \cdot H$$

The **2D Discrete Fourier Transform (DFT)** decomposes an image into its constituent spatial frequencies. Low frequencies represent smooth, gradual changes; high frequencies represent edges and fine detail.

Key NumPy functions:
- `np.fft.fft2(img)` — compute the 2D DFT
- `np.fft.fftshift(F)` — move the DC component (average brightness) to the center
- `np.fft.ifft2(F)` — inverse DFT (back to spatial domain)

### Magnitude spectrum

To visualize the frequency content, we plot the **log-magnitude** of the DFT: $\log(1 + |F(u,v)|)$.

The log scaling compresses the huge dynamic range so that both the bright DC component and the faint high-frequency components are visible.

In [ ]:
show_spectrum(img_gray, title="Grayscale parrot")

### How image content maps to frequency content

Smooth regions concentrate energy at the **center** (low frequencies), while edges and textures spread energy **outward** (high frequencies). Let’s compare the original with a heavily blurred version.

In [ ]:
smooth = correlate(img_gray.astype(np.float64),
                   make_gaussian_kernel(31, 5.0), mode="constant", cval=0)
smooth = np.clip(smooth, 0, 255).astype(np.uint8)

show_spectrum(img_gray, title="Original")
show_spectrum(smooth, title="Heavily smoothed")

### Low-pass filtering in frequency domain

We can blur an image directly in the frequency domain:
1. Compute 2D DFT
2. Multiply by a **low-pass mask** (keeps center, blocks edges)
3. Inverse DFT back to spatial domain

The result is equivalent to spatial convolution with a smoothing filter.

In [ ]:
# Compute 2D DFT
F = np.fft.fft2(img_gray.astype(np.float64))
F_shifted = np.fft.fftshift(F)

# Create a circular low-pass mask
H, W = img_gray.shape
cy, cx = H // 2, W // 2
Y, X = np.ogrid[:H, :W]
radius = 30
mask_lp = ((X - cx)**2 + (Y - cy)**2 <= radius**2).astype(np.float64)

# Apply mask and inverse FFT
F_filtered = F_shifted * mask_lp
result_lp = np.fft.ifft2(np.fft.ifftshift(F_filtered)).real
result_lp = np.clip(result_lp, 0, 255).astype(np.uint8)

fig, axes = plt.subplots(1, 4, figsize=(16, 4))
axes[0].imshow(img_gray, cmap="gray", vmin=0, vmax=255)
axes[0].set_title("Original")
axes[1].imshow(np.log1p(np.abs(F_shifted)), cmap="gray")
axes[1].set_title("Spectrum")
axes[2].imshow(mask_lp, cmap="gray")
axes[2].set_title(f"Low-pass mask (r={radius})")
axes[3].imshow(result_lp, cmap="gray", vmin=0, vmax=255)
axes[3].set_title("Low-pass filtered")
for ax in axes:
    ax.axis("off")
plt.tight_layout()
plt.show()

### High-pass filtering in frequency domain

The complement of low-pass is high-pass: $H_{\text{HP}} = 1 - H_{\text{LP}}$

High-pass filtering keeps edges and fine detail while removing smooth regions.

In [ ]:
mask_hp = 1 - mask_lp

F_hp = F_shifted * mask_hp
result_hp = np.fft.ifft2(np.fft.ifftshift(F_hp)).real

# Rescale for display (high-pass output has negative values)
hp_min, hp_max = result_hp.min(), result_hp.max()
result_hp_display = ((result_hp - hp_min) / (hp_max - hp_min) * 255).astype(np.uint8)

fig, axes = plt.subplots(1, 3, figsize=(12, 4))
axes[0].imshow(img_gray, cmap="gray", vmin=0, vmax=255)
axes[0].set_title("Original")
axes[1].imshow(mask_hp, cmap="gray")
axes[1].set_title("High-pass mask")
axes[2].imshow(result_hp_display, cmap="gray", vmin=0, vmax=255)
axes[2].set_title("High-pass filtered (edges)")
for ax in axes:
    ax.axis("off")
plt.tight_layout()
plt.show()

### Frequency response of spatial kernels

We can compute the frequency response of any spatial kernel by zero-padding it to image size and taking its DFT. This reveals **why Gaussian beats Box**:

- **Gaussian**: smooth, clean rolloff in frequency (no oscillations)
- **Box**: sinc-like response with sidelobes that cause **ringing artifacts**

In [ ]:
def kernel_freq_response(kernel, shape):
    """Zero-pad kernel to shape and return centered magnitude spectrum."""
    padded = np.zeros(shape, dtype=np.float64)
    kH, kW = kernel.shape
    padded[:kH, :kW] = kernel
    F = np.fft.fftshift(np.fft.fft2(padded))
    return np.abs(F)

box_k = make_box_kernel(15)
gauss_k = make_gaussian_kernel(15, sigma=3.0)

box_resp = kernel_freq_response(box_k, img_gray.shape)
gauss_resp = kernel_freq_response(gauss_k, img_gray.shape)

# 1D cross-section through center
center = img_gray.shape[0] // 2
freq_axis = np.arange(img_gray.shape[1]) - img_gray.shape[1] // 2

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
axes[0].plot(freq_axis, box_resp[center, :], label="Box 15\u00d715")
axes[0].plot(freq_axis, gauss_resp[center, :], label="Gaussian \u03c3=3")
axes[0].set_title("Frequency response (1D cross-section)")
axes[0].set_xlabel("Frequency")
axes[0].set_ylabel("Magnitude")
axes[0].legend()
axes[0].set_xlim([-100, 100])

axes[1].imshow(np.log1p(box_resp), cmap="gray")
axes[1].set_title("Box 2D response (log)")
axes[1].axis("off")
plt.tight_layout()
plt.show()

### Exercises

**Exercise 4.1:** Experiment with different **low-pass cutoff radii** (10, 30, 60, 100). For each radius, create a circular mask, apply it in the frequency domain, and display the result. Show all four filtered images in a single row.

*Steps:*
1. Compute `F_shifted = np.fft.fftshift(np.fft.fft2(img_gray.astype(np.float64)))`
2. For each radius `r`, create a mask: `((X - cx)**2 + (Y - cy)**2 <= r**2).astype(np.float64)` — reuse `Y, X, cy, cx` from the demo above
3. Filter: `np.fft.ifft2(np.fft.ifftshift(F_shifted * mask)).real`
4. Clip to [0, 255] and cast to `uint8`

What happens as the radius increases? How does this relate to the kernel size of spatial smoothing?

In [ ]:
# YOUR CODE HERE

**Exercise 4.2:** Compute the magnitude spectrum of the **original** grayscale image and of a **gamma-corrected** version ($\gamma = 0.3$). Display both spectra side by side using `show_spectrum`.

*Hint:* Apply gamma correction from Week 2: `corrected = (255 * (img_gray / 255.0) ** 0.3).astype(np.uint8)`. How does brightening dark regions affect the frequency content?

This connects to **Week 2 Section 5 (Gamma Correction)** — you are observing gamma’s effect in the frequency domain, not just the spatial domain.

In [ ]:
# YOUR CODE HERE

---
## 5. Sharpening Filters

Sharpening enhances edges and fine detail. The key insight:

$$\text{High-pass} = \text{Original} - \text{Low-pass (blurred)}$$

Therefore:

$$\text{Sharpened} = \text{Original} + \alpha \cdot \text{High-pass}$$

Two common approaches:
1. **Laplacian**: a single kernel that directly computes the second derivative (edges)
2. **Unsharp masking**: blur the image, subtract the blur to get edges, add them back

### Laplacian filter

The Laplacian kernel detects edges by computing the **second derivative** of the image. Its weights sum to **zero**, so flat regions produce zero output — only intensity changes survive.

Two variants:
- **4-connected**: $\begin{bmatrix}0 & 1 & 0\\1 & -4 & 1\\0 & 1 & 0\end{bmatrix}$ — horizontal & vertical edges
- **8-connected**: $\begin{bmatrix}1 & 1 & 1\\1 & -8 & 1\\1 & 1 & 1\end{bmatrix}$ — includes diagonal edges

In [ ]:
lap4 = np.array([[0,  1, 0],
                 [1, -4, 1],
                 [0,  1, 0]], dtype=np.float64)

lap8 = np.array([[1,  1, 1],
                 [1, -8, 1],
                 [1,  1, 1]], dtype=np.float64)

fig, axes = plt.subplots(1, 2, figsize=(6, 3))
show_kernel(lap4, title="Laplacian (4-connected)", ax=axes[0])
show_kernel(lap8, title="Laplacian (8-connected)", ax=axes[1])
plt.tight_layout()
plt.show()

print(f"4-connected sum: {lap4.sum():.0f}")
print(f"8-connected sum: {lap8.sum():.0f}")

In [ ]:
edges4 = correlate(img_gray.astype(np.float64), lap4, mode="constant", cval=0)
edges8 = correlate(img_gray.astype(np.float64), lap8, mode="constant", cval=0)

def rescale_for_display(img):
    """Rescale a float image to [0, 255] uint8 for display."""
    lo, hi = img.min(), img.max()
    return ((img - lo) / (hi - lo) * 255).astype(np.uint8)

show_images(img_gray, rescale_for_display(edges4), rescale_for_display(edges8),
            titles=["Original", "Laplacian 4-connected", "Laplacian 8-connected"])

### Sharpening with the Laplacian

To sharpen, **subtract** the Laplacian (edge map) from the original:

$$\text{Sharpened} = \text{Original} - \text{Laplacian}$$

(We subtract because the standard Laplacian has a **negative** center weight. Equivalently, some formulations negate the kernel and add instead.)

In [ ]:
sharpened_lap = img_gray.astype(np.float64) - edges4
sharpened_lap = np.clip(sharpened_lap, 0, 255).astype(np.uint8)

show_images(img_gray, rescale_for_display(edges4), sharpened_lap,
            titles=["Original", "Laplacian edges", "Sharpened (orig \u2212 Laplacian)"])

### Unsharp masking

A more flexible sharpening method with a tunable strength parameter $\alpha$:

$$\text{Sharpened} = \text{Original} + \alpha \cdot (\text{Original} - \text{Blurred})$$

The term $(\text{Original} - \text{Blurred})$ extracts the high-frequency detail. Adding it back at strength $\alpha$ enhances edges.

In [ ]:
def unsharp_mask(img, sigma=3.0, alpha=1.0):
    """Apply unsharp masking to a grayscale image."""
    size = int(6 * sigma + 1) | 1
    kernel = make_gaussian_kernel(size, sigma)
    blurred = correlate(img.astype(np.float64), kernel, mode="constant", cval=0)
    detail = img.astype(np.float64) - blurred
    sharpened = img.astype(np.float64) + alpha * detail
    return np.clip(sharpened, 0, 255).astype(np.uint8)

In [ ]:
alphas = [0.5, 1.0, 2.0, 4.0]
results = [unsharp_mask(img_gray, sigma=3.0, alpha=a) for a in alphas]

show_images(img_gray, *results,
            titles=["Original"] + [f"\u03b1 = {a}" for a in alphas])

Let’s also visualize the **detail** (high-pass) image that unsharp masking extracts.

In [ ]:
size = int(6 * 3.0 + 1) | 1
k = make_gaussian_kernel(size, 3.0)
blurred_for_detail = correlate(img_gray.astype(np.float64), k, mode="constant", cval=0)
detail = img_gray.astype(np.float64) - blurred_for_detail

show_images(img_gray,
            np.clip(blurred_for_detail, 0, 255).astype(np.uint8),
            rescale_for_display(detail),
            titles=["Original", "Blurred (low-pass)", "Detail (high-pass)"])

### Exercises

**Exercise 5.1:** Apply **unsharp masking** to the **color** image. Process each channel independently using the `unsharp_mask` function with $\sigma=2, \alpha=1.5$. Recombine the channels and display original vs sharpened.

*Steps:*
1. Split: `channels = [img_color[:,:,c] for c in range(3)]`
2. Apply: `sharp_channels = [unsharp_mask(ch, sigma=2.0, alpha=1.5) for ch in channels]`
3. Stack: `result = np.stack(sharp_channels, axis=2)`
4. Display with `show_images`

In [ ]:
# YOUR CODE HERE

**Exercise 5.2:** Create a **before/after comparison** that combines Weeks 2 and 3:
1. Start with the grayscale image
2. Apply **gamma correction** with $\gamma = 0.7$ to brighten dark regions (Week 2)
3. Apply **unsharp masking** with $\sigma=2, \alpha=1.0$ to sharpen (Week 3)
4. Display three images: Original, After gamma, After gamma + sharpening

*Hint:* Apply gamma first: `bright = (255 * (img_gray / 255.0) ** 0.7).astype(np.uint8)`, then sharpen with `unsharp_mask(bright, sigma=2.0, alpha=1.0)`.

This demonstrates a realistic **image enhancement pipeline**: first fix brightness, then sharpen details.

In [ ]:
# YOUR CODE HERE

**Exercise 5.3:** Apply the 8-connected Laplacian kernel to the grayscale image and display the edge map. Then compute the **histogram** of the edge image (before rescaling) and plot it.

What is the shape of this histogram? Why does it look so different from the histogram of the original image?

*Hint:* Use `correlate` with `lap8`. For the histogram, work with the raw float output. Cast to `int16` first:
```python
edges = correlate(img_gray.astype(np.float64), lap8, mode="constant", cval=0)
edges_int = edges.astype(np.int16)
h, bins = np.histogram(edges_int.ravel(), bins=100)
plt.fill_between(bins[:-1], h, alpha=0.7)
```

This connects to **Week 2 Section 7 (Histograms)** — you are analyzing histogram shape to understand what a filter does to the intensity distribution.

In [ ]:
# YOUR CODE HERE